In [1]:
# Dependencies
import numpy as np
import pandas as pd
import pandas as pd
import os
from tensorflow.contrib.tensorboard.plugins import projector
from sklearn.manifold import TSNE
from collections import Counter
from six.moves import cPickle
import gensim.models.word2vec as w2v
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import multiprocessing
import os
import sys
import io
import re
import json
import nltk
import string
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer

In [2]:
training_drop_na_df = pd.read_csv(r"C:\Users\jjtre\Documents\UCSD Coding Bootcamp Data Science\Final Project\testing_cleaned_tweets_stopword_filter_dropnan.csv", encoding='utf-8')
training_drop_na_df.head()

,tweet_polarity,tweet_text,clean_tweets
0,0,"@switchfoot http://twitpic.com/2y1zl - Awww, t...",awww bummer shoulda got david carr third day
1,0,is upset that he can't update his Facebook by ...,upset update facebook texting might cry result...
2,0,@Kenichan I dived many times for the ball. Man...,dived many times ball managed save 50 rest go ...
3,0,my whole body feels itchy and like its on fire,whole body feels itchy like fire
4,0,"@nationwideclass no, it's not behaving at all....",behaving mad see


In [3]:
training_drop_na_df['total_words'] = training_drop_na_df['clean_tweets'].str.split().str.len()
training_drop_na_df.head()

,tweet_polarity,tweet_text,clean_tweets,total_words
0,0,"@switchfoot http://twitpic.com/2y1zl - Awww, t...",awww bummer shoulda got david carr third day,8
1,0,is upset that he can't update his Facebook by ...,upset update facebook texting might cry result...,11
2,0,@Kenichan I dived many times for the ball. Man...,dived many times ball managed save 50 rest go ...,10
3,0,my whole body feels itchy and like its on fire,whole body feels itchy like fire,6
4,0,"@nationwideclass no, it's not behaving at all....",behaving mad see,3


In [4]:
X = training_drop_na_df[['clean_tweets', 'total_words']]
y = training_drop_na_df['tweet_polarity']
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)

In [7]:
from sklearn.base import BaseEstimator, TransformerMixin
class TextSelector(BaseEstimator, TransformerMixin):
    def __init__(self, field):
        self.field = field
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        return X[self.field]
class NumberSelector(BaseEstimator, TransformerMixin):
    def __init__(self, field):
        self.field = field
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        return X[[self.field]]

In [9]:
import nltk
def Tokenizer(str_input):
    words = re.sub(r"[^A-Za-z0-9\-]", " ", str_input).lower().split()
    porter_stemmer=nltk.PorterStemmer()
    words = [porter_stemmer.stem(word) for word in words]
    return words

In [11]:
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import TruncatedSVD
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
classifier = Pipeline([
    ('features', FeatureUnion([
        ('text', Pipeline([
            ('colext', TextSelector('clean_tweets')),
            ('tfidf', TfidfVectorizer(tokenizer=Tokenizer,
                     min_df=.0025, max_df=0.25, ngram_range=(1,3))),
            ('svd', TruncatedSVD(algorithm='randomized', n_components=300)), #for XGB
        ])),
        ('words', Pipeline([
            ('wordext', NumberSelector('total_words')),
            ('wscaler', StandardScaler()),
        ])),
    ])),
    ('clf', XGBClassifier(max_depth=3, n_estimators=300, learning_rate=0.1)),
#    ('clf', RandomForestClassifier()),
    ])

In [12]:
classifier.fit(X_train, y_train)
preds = classifier.predict(X_test)

C:\Users\jjtre\Anaconda3\envs\PythonData\lib\site-packages\sklearn\preprocessing\data.py:645: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
C:\Users\jjtre\Anaconda3\envs\PythonData\lib\site-packages\sklearn\base.py:467: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  return self.fit(X, y, **fit_params).transform(X)
C:\Users\jjtre\Anaconda3\envs\PythonData\lib\site-packages\sklearn\pipeline.py:451: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)


In [15]:
from sklearn.metrics import accuracy_score, precision_score, classification_report, confusion_matrix
print("Accuracy:", accuracy_score(y_test, preds))
#print("Precision:", precision_score(y_test, preds))
print(classification_report(y_test, preds))
print(confusion_matrix(y_test, preds))

Accuracy: 0.7127083479850904
              precision    recall  f1-score   support

           0       0.72      0.69      0.71    199081
           4       0.70      0.74      0.72    199051

   micro avg       0.71      0.71      0.71    398132
   macro avg       0.71      0.71      0.71    398132
weighted avg       0.71      0.71      0.71    398132

[[136740  62341]
 [ 52039 147012]]


In [17]:
import joblib
filename = 'twitter_XGBoost_hypertuned.sav'
joblib.dump(classifier, filename)

['twitter_XGBoost_hypertuned.sav']

In [20]:
# Save the model
import joblib
filename = 'twitter_XGBoost_hypertuned.h5'
joblib.dump(classifier, filename)

['twitter_XGBoost_hypertuned.h5']